##### Wybór zbioru testowego, wczytanie anotacji GT, wspólne funkcje pomocnicze, definicja słownika klas i promptów.

### 0. Inicjalizacja środowiska i ścieżek

In [31]:
from pathlib import Path
import pandas as pd

"""
Inicjalizuje ścieżki i katalogi wyjściowe dla benchmarku detektorów.

Założenia:
- PROJECT_ROOT wskazuje na katalog główny repozytorium MetaLogic.
- Dane testowe znajdują się w podkatalogu inputs/.
- Wyniki detektorów i metryk zapisujemy w outputs/detector_eval/.
"""

PROJECT_ROOT = Path.cwd().resolve()
DATA_DIR = PROJECT_ROOT / "data"
IMAGES_DIR = PROJECT_ROOT / "inputs"
GT_DIR = DATA_DIR / "gt_annotations"      # Ground truth (opcjonalnie)
EVAL_OUTPUT_DIR = PROJECT_ROOT / "outputs" / "detector_eval"

EVAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

### 1. Konfiguracja klas, promptów i modeli

In [32]:
"""
Definiuje wspólny słownik klas oraz prompty tekstowe dla modeli open-vocabulary.

class_norm – nazwy klas używane w benchmarku (po normalizacji, bez polskich znaków).
prompts     – mapowanie class_norm → listę promptów tekstowych dla modeli takich jak OWL-ViT.
detectors   – lista nazw detektorów branych pod uwagę w benchmarku.
"""

CLASS_NORM = [
    "syrena",
    "maluch",
    "kosz_na_smieci",
    "pies",
    "kura",
    "kot",
    "kaluza",
    "czepek_kucharza",
    "wozek_dzieciecy",
    "zabytkowa_kamienica",
    "szyld_sklepowy",
    "zrujnowany_mur",
    "zrujnowany_dom",
    "gablota_fotograficzna",
    "zabytkowa_wieza",
    "latarnia_uliczna",
    "ulica_w_deszczu",
    "ulica_zima",
    "stare_miasto",
    "kamienice_miejskie",
    "tradycyjna_architektura",
    "boniowanie",
    "stella",
    "glowica_jonska",
    "portyk",
    "kolumny_portykowe",
    "kosciol",
    "balkon",
    "pochod",
    "demonstracja",
    "zgromadzenie",
    "wnetrze_sklepu",
    "witryna_sklepowa",
    "kolejka_przed_sklepem",
    "trolejbus",
]

PROMPTS_BY_CLASS = {
    "syrena": ["Syrena 105 car", "Syrena car"],
    "maluch": ["Fiat 126p car", "small Fiat 126p"],
    "kosz_na_smieci": ["trash bin", "garbage can", "street trash bin"],
    "pies": ["dog", "pet dog"],
    "kura": ["chicken", "hen"],
    "kot": ["cat"],
    "kaluza": ["puddle", "water puddle on street"],
    "czepek_kucharza": ["chef hat", "cook hat"],
    "wozek_dzieciecy": ["baby stroller", "pram", "child stroller"],
    "zabytkowa_kamienica": ["historic tenement house", "old townhouse"],
    "szyld_sklepowy": ["shop sign", "store sign"],
    "zrujnowany_mur": ["decayed wall", "ruined wall"],
    "zrujnowany_dom": ["ruined house", "derelict house"],
    "gablota_fotograficzna": ["photo display case", "street photo display"],
    "zabytkowa_wieza": ["historic tower", "old tower"],
    "latarnia_uliczna": ["street lamp", "street light"],
    "ulica_w_deszczu": ["street in rain", "rainy street"],
    "ulica_zima": ["winter street", "snowy street"],
    "stare_miasto": ["old town street", "historic old town"],
    "kamienice_miejskie": ["city tenement houses", "row of townhouses"],
    "tradycyjna_architektura": ["traditional architecture building facade"],
    "boniowanie": ["rusticated facade", "rustication on wall"],
    "stella": ["stone stele monument", "memorial stele"],
    "glowica_jonska": ["Ionic capital", "Ionic column capital"],
    "portyk": ["portico entrance", "classical portico"],
    "kolumny_portykowe": ["portico columns", "classical columns"],
    "kosciol": ["church building", "historic church"],
    "balkon": ["balcony", "building balcony"],
    "pochod": ["street procession", "parade march"],
    "demonstracja": ["street protest", "demonstration crowd"],
    "zgromadzenie": ["crowd gathering", "public assembly"],
    "wnetrze_sklepu": ["shop interior", "store interior"],
    "witryna_sklepowa": ["shop window", "storefront window"],
    "kolejka_przed_sklepem": ["queue in front of shop", "line of people outside store"],
    "trolejbus": ["trolleybus", "electric trolley bus"],
}

DETECTORS = [
    "yolo",
    "lvis",
    "owl_vit",
    "detr",    # opcjonalnie
]

### 2. Lista obrazów do benchmarku

In [33]:
"""
Buduje listę obrazów testowych NA NOWO na podstawie IMAGES_DIR.

- ignoruje ewentualny stary eval_file_list.csv,
- zawsze skanuje bieżący katalog IMAGES_DIR,
- zapisuje nowy CSV z kolumną 'file_name'.
"""

TEST_LIST_CSV = EVAL_OUTPUT_DIR / "eval_file_list.csv"

image_exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff"}
rows = []

for p in sorted(IMAGES_DIR.rglob("*")):
    if p.suffix.lower() in image_exts:
        rows.append({"file_name": p.name})

if not rows:
    raise ValueError(
        f"Nie znaleziono żadnych obrazów w katalogu: {IMAGES_DIR.as_posix()}\n"
        f"Sprawdź, czy ścieżka IMAGES_DIR jest poprawna i czy katalog 'inputs/' zawiera pliki "
        f"o rozszerzeniach: {sorted(image_exts)}."
    )

df_files = pd.DataFrame(rows)
df_files.to_csv(TEST_LIST_CSV, index=False)

df_files.head()

,file_name
0,0000.jpg
1,000202.jpg
2,000238.jpg
3,0003.jpg
4,000345.jpg


### 3. Wczytanie annotacji ground truth (opcjonalne)

In [34]:
"""
Wczytuje annotacje ground truth dla wybranego podzbioru obrazów.

Założenia:
- GT zapisane w jednym CSV (np. format uproszczony, nie COCO),
- kolumny: file_name, class_norm, x_min, y_min, x_max, y_max.
"""

GT_CSV = GT_DIR / "gt_boxes.csv"

if GT_CSV.exists():
    df_gt = pd.read_csv(GT_CSV)
else:
    df_gt = pd.DataFrame(columns=["file_name", "class_norm", "x_min", "y_min", "x_max", "y_max"])

df_gt.head()

,file_name,class_norm,x_min,y_min,x_max,y_max


### 4. Podsumowanie konfiguracji benchmarku

In [35]:
"""
Wyświetla podstawowe informacje o konfiguracji benchmarku:
- liczba obrazów,
- liczba annotacji GT,
- lista klas i detektorów.
"""

print("Liczba obrazów w zbiorze testowym:", len(df_files))
print("Liczba annotacji GT:", len(df_gt))
print("\nKlasy (class_norm):", CLASS_NORM)
print("Detektory:", DETECTORS)

Liczba obrazów w zbiorze testowym: 134
Liczba annotacji GT: 0

Klasy (class_norm): ['syrena', 'maluch', 'kosz_na_smieci', 'pies', 'kura', 'kot', 'kaluza', 'czepek_kucharza', 'wozek_dzieciecy', 'zabytkowa_kamienica', 'szyld_sklepowy', 'zrujnowany_mur', 'zrujnowany_dom', 'gablota_fotograficzna', 'zabytkowa_wieza', 'latarnia_uliczna', 'ulica_w_deszczu', 'ulica_zima', 'stare_miasto', 'kamienice_miejskie', 'tradycyjna_architektura', 'boniowanie', 'stella', 'glowica_jonska', 'portyk', 'kolumny_portykowe', 'kosciol', 'balkon', 'pochod', 'demonstracja', 'zgromadzenie', 'wnetrze_sklepu', 'witryna_sklepowa', 'kolejka_przed_sklepem', 'trolejbus']
Detektory: ['yolo', 'lvis', 'owl_vit', 'detr']
